# Exercises

## Acquire (`acquire.py`)

### Zillow
**For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.**

**You will have a** 
`zillow.ipynb` 
**file and a helper file for each section in the pipeline.**

In [11]:
###############################################################################
### python imports                                                          ###
###############################################################################

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LassoCV

import warnings
warnings.filterwarnings("ignore")

# from math import sqrt
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [12]:
###############################################################################
### local imports                                                           ###
###############################################################################

import acquire as acq
import prepare as prep

from wrangle_zillow import get_zillow_data
from debug import local_settings, timeifdebug, timeargsifdebug, frame_splain
from dfo import DFO, set_dfo


#### Acquire & Summarize

1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

   - ***Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for*** 
`airconditioningtypeid`
***.***
   - Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
   - Only include properties that include a latitude and longitude value.

In [2]:
local_settings.splain=True
local_settings.debug=True
splain = local_settings.splain
debug = local_settings.debug

In [3]:
dfo = set_dfo(get_zillow_data(), splain=True)

starting get_zillow_data
starting sql_df
starting get_db_url
ending get_db_url ; time: 1.0967254638671875e-05
starting check_df
starting frame_splain
ending frame_splain ; time: 1.6927719116210938e-05
ending check_df ; time: 0.14870691299438477
ending sql_df ; time: 118.18951296806335
ending get_zillow_data ; time: 118.18963813781738
starting set_dfo
ending set_dfo ; time: 9.059906005859375e-06


2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [4]:
df = dfo.df

In [39]:
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 72 columns):
id                              77381 non-null int64
parcelid                        77381 non-null int64
airconditioningtypeid           24953 non-null float64
airconditioningdesc             24953 non-null object
architecturalstyletypeid        206 non-null float64
architecturalstyledesc          206 non-null object
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingclasstypeid             15 non-null float64
buildingclassdesc               15 non-null object
buildingqualitytypeid           49672 non-null float64
calculatedbathnbr               76772 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
decktypeid                      614 non-null float64
finishedfloor1squarefeet        6023 non-null float64
finishedsquarefeet12            73

In [5]:
frame_splain(dfo.df, title='Zillow Data', splain=True)

starting frame_splain
ZILLOW DATA SHAPE:
(77381, 72) 

ZILLOW DATA INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 72 columns):
id                              77381 non-null int64
parcelid                        77381 non-null int64
airconditioningtypeid           24953 non-null float64
airconditioningdesc             24953 non-null object
architecturalstyletypeid        206 non-null float64
architecturalstyledesc          206 non-null object
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingclasstypeid             15 non-null float64
buildingclassdesc               15 non-null object
buildingqualitytypeid           49672 non-null float64
calculatedbathnbr               76772 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
decktypeid                      614 non-null float64
finishedfloor1s

3: Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

||num_rows_missing|pct_rows_missing|
|:---|---:|---:|
|parcelid|0|0.000000|
|airconditioningtypeid|29041|0.535486|
|architecturalstyletypeid|54232|0.999982|


In [104]:
@timeifdebug
def get_column_values_stats(
        df, 
        get_cols=[], 
        max_uniques=10, 
        target_col='',
        limit_to_max=False,
        **kwargs
        ):
    '''
    get_column_values_stats(
        df, 
        get_cols=[], 
        max_uniques=10, 
        target_col='', 
        limit_to_max=False,
        **kwargs
        )
    RETURNS summary dataframe

    Receives dataframe as input, examines all columns defined as objects, and
    returns a summary report with column name as its index.

    Report shows the following information about the columns of the dataframe:
        cols: column name
        dtypes: data type
        row_values: number of rows with non-null values 
        rows_missing: number of rows with missing values
        pctg_missing: percentage of rows with missing values
        nunuiques: number of unique values 
        uniques: list of unique values if the unique value count is less than 
            or equal to max_uniques.
        
    If the input dataframe contains the target column, enter that name as the 
    target_col argument and it will be removed from the analysis.
    
    If limit_to_max is True (default), the resulting dataframe will only show
    columns with no more unique values than specified in max_uniques.
    
    NOTE: This function can be used to identify and then re-type catergorical
    columns. Run the function with the following parameters, change the data 
    types of know categorical values to 'category', then re-run this function, 
    gradually increasing the max_uniques value until the desired limit for 
    categoricals has been met.
    
    get_column_values_stats(
        df, 
        get_cols=df.columns[df.dtypes!='category'],
        max_uniques=5,
        limit_to_max=True
        )
    '''
    # get rowcount
    num_recs = len(df)
    
    # if no columns presented, get all columns from df
    use_cols = df.columns if len(get_cols) == 0 else get_cols
    # ensure use_cols are actually present in df
    cols = [col for col in use_cols if col in df.columns]
    
    # make df for all columns, exclude target column if passed
    df_cols = pd.DataFrame(cols, columns=['cols'])
    df_cols = df_cols[df_cols.cols != target_col]
    
    # get statistics on all columns
    df_cols['dtype'] = df_cols.cols.apply(lambda x: df[x].dtype)
    df_cols['row_values'] = df_cols.cols.apply(lambda x: df[x].count())
    df_cols['rows_missing'] = num_recs - df_cols.row_values
    df_cols['pctg_missing'] = df_cols.rows_missing / num_recs
    df_cols['nuniques'] = df_cols.cols.apply(lambda x: df[x].nunique())
    
    # get unique valeues for columns within unique value limits
    df_cats = df_cols[df_cols.nuniques <= max_uniques]
    df_cats['uniques'] = df_cats.cols.apply(lambda x: df[x].unique())
    
    # merge and set index
    join_type = 'inner' if limit_to_max else 'left'
    df_cols = df_cols.join(df_cats.uniques, how=join_type)
    return df_cols.set_index('cols')

In [108]:
get_column_values_stats(df, get_cols=df.columns[df.dtypes!='category'],max_uniques=15,limit_to_max=False)

starting get_row_values_stats
left
ending get_row_values_stats ; time: 0.40561985969543457


,dtype,row_values,rows_missing,pctg_missing,nuniques,uniques
cols,,,,,,
id,int64,77381,0,0.000000,77381,NaN
parcelid,int64,77381,0,0.000000,77381,NaN
airconditioningtypeid,float64,24953,52428,0.677531,5,"[nan, 1.0, 13.0, 5.0, 11.0, 9.0]"
airconditioningdesc,object,24953,52428,0.677531,5,"[nan, Central, Yes, None, Wall Unit, Refrigera..."
architecturalstyletypeid,float64,206,77175,0.997338,5,"[nan, 7.0, 8.0, 21.0, 3.0, 2.0]"
architecturalstyledesc,object,206,77175,0.997338,5,"[nan, Contemporary, Conventional, Ranch/Ramble..."
basementsqft,float64,50,77331,0.999354,43,NaN
bathroomcnt,float64,77381,0,0.000000,22,NaN
bedroomcnt,float64,77381,0,0.000000,16,NaN


4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

num_cols_missing|pct_cols_missing|num_rows
:---|---:|---:
23|38.333|108
24|40.000|123
25|41.667|5280

#### Prepare

1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

   - The input:
       - A dataframe
       - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
       - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
   - The output:
       - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
   - hint:
       - Look up the dropna documentation.
       - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
       - Make use of inplace, i.e. inplace=True/False.

In [35]:
pd.DataFrame(df.info())#.loc(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 72 columns):
id                              77381 non-null int64
parcelid                        77381 non-null int64
airconditioningtypeid           24953 non-null float64
airconditioningdesc             24953 non-null object
architecturalstyletypeid        206 non-null float64
architecturalstyledesc          206 non-null object
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingclasstypeid             15 non-null float64
buildingclassdesc               15 non-null object
buildingqualitytypeid           49672 non-null float64
calculatedbathnbr               76772 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
decktypeid                      614 non-null float64
finishedfloor1squarefeet        6023 non-null float64
finishedsquarefeet12            73

""


3. Decide how to handle the remaining missing values:
   - Fill with constant value.
   - Impute with mean, median, mode.
   - Drop row/column

In [92]:
df_sqft = df.loc[:,['calculatedfinishedsquarefeet',
'finishedfloor1squarefeet',
'finishedsquarefeet6',
'finishedsquarefeet12',
'finishedsquarefeet13',
'finishedsquarefeet15',
'finishedsquarefeet50'
]]

#### `wrangle_zillow.py`

Functions of the work above needed to acquire and prepare a new sample of data.

In [119]:
df_sqft[df_sqft.finishedfloor1squarefeet.isna() & (df_sqft.finishedsquarefeet12.isna()==False)]

,calculatedfinishedsquarefeet,finishedfloor1squarefeet,finishedsquarefeet6,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50
38,2220.0,NaN,NaN,2220.0,NaN,NaN,NaN
58,2376.0,NaN,NaN,2376.0,NaN,NaN,NaN
59,1312.0,NaN,NaN,1312.0,NaN,NaN,NaN
60,2962.0,NaN,NaN,2962.0,NaN,NaN,NaN
61,738.0,NaN,NaN,738.0,NaN,NaN,NaN
62,3039.0,NaN,NaN,3039.0,NaN,NaN,NaN
63,1290.0,NaN,NaN,1290.0,NaN,NaN,NaN
64,2540.0,NaN,NaN,2540.0,NaN,NaN,NaN
65,1435.0,NaN,NaN,1435.0,NaN,NaN,NaN
66,1146.0,NaN,NaN,1146.0,NaN,NaN,NaN
